# **Using ChatGPT with Python**

## **Securely Handling Sensitive Data in Colab**

To use OpenAI's GPT models the API key needs to be set up.

Before being able to use the Chat Completions API
endpoint we need to authenticate ourselves using the **API key**. Having finished the preparatory assignments you should have your own API key which you can use during the course as well as the post assignment.

When working with sensitive information like API keys or passwords in Google Colab, it's crucial to handle data securely. Two common approaches for this are using **Colab's Secrets Manager**, which stores and retrieves secrets without exposing them in the notebook, and `getpass`, a Python function that securely prompts users to input secrets during runtime without showing them. Both methods help ensure your sensitive data remains protected.

### **Option 1: Using Google Colab Secrets Manager**

Google Colab provides an integrated Secrets Manager, allowing you to securely store and retrieve sensitive information such as API keys or authentication tokens without hardcoding them in your notebook.

**Step 1: Store Your Secret in Colab**

1.   In the Colab notebook, navigate to the left sidebar.
2.   Click on the **“Secrets”** tab (represented by a key icon).
3. Add your secret by clicking on **“+ Add a new secret”**. For example, you might add a secret called `OPENAI_API_KEY` with the value of your API key.

**Step 2: Access the Secret in Your Notebook**

Once you've added a secret, you can easily access it from within the notebook.

`OPENAI_API_KEY` is the name of the secret you've added in the Colab Secrets Manager. It will be retrieved securely without having to expose the key in the notebook.


In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

### **Option 2: Using Python's `getpass` for Secret Input**

Alternatively, the `getpass` module allows you to securely input secrets (e.g., passwords or API keys) during runtime, making sure they're not visible in the notebook output.

Here, the `getpass.getpass` function prompts the user to enter the secret without displaying it as they type, ensuring that sensitive data isn't exposed.

In [ ]:
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass()

## **Installing OpenAI**

In this notebook you will learn how to use ChatGPT with Python for various tasks such as **text generation**, **speech-to-text**,  **text-to-speech**, and  **image generation**.

In [ ]:
!pip install -q openai

## **Text Generation**

OpenAI's text generation **models** (often called generative pre-trained transformers or large language models) have been trained to understand natural language, code, and images. The latest models, `gpt-4o` (large model), `gpt-4o-mini` (small model), and the `o1`  familty of models (reasoning model) are accessed through the **chat completions API endpoint**.

> For an overview of the different models for text generation you can visit https://platform.openai.com/docs/models.

The models provide text outputs in response to  inputs given to them. The inputs to these models are also referred to as **"prompts"**. Designing a prompt is essentially how you “program” a large language model, usually by providing instructions or some examples of how to successfully complete a task.

To use one of these models via the OpenAI API, you can send a request containing the inputs and your API key, and receive a response containing the model's output.




### **Chat Completions API**

The Chat Completions API allows you to interact with chat models by providing a list of messages as input. The model processes the conversation history and returns a message generated based on the input, enabling dynamic and context-aware conversations.

To use the Chat Completions API, you first need to import the necessary class and initialize the OpenAI client.

In [ ]:
from openai import OpenAI
client = OpenAI()

After successful authentication the Chat Completions API can be used. The API call has two required inputs:

* `model`: the name of the model you want to use (e.g., `gpt-4o`, `gpt-4o-mini`)
* `messages`: a list of message objects, where each object has two required fields:
   * `role`: the role of the messenger (either developer, user, or assistant)
   * `content`: the content of the message (e.g., Write me a beautiful poem)

There are additional optional parameters:

* `temperature`: the temperature parameter allows you to balance randomness and determinism in the output. Low temperature values make the output more focussed and deterministic, while a high temperature increases randomness in the output and produce more unexpected outputs.

Typically, a conversation will start with a system message that tells the assistant how to behave, followed by alternating user and assistant messages, but you are not required to follow this format.

While the **developer message** helps set the behavior of the assistant, the **user messages** provide requests or comments for the assistant to respond to, and  **assistant messages** store previous assistant responses, but can also be written by you to give examples of desired behavior.


An example Chat Completions API call:

In [ ]:
#Example with a system message
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "developer", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Explain generative AI to a 6 year old."},

  ]
)

In [ ]:
print(response)

The response format of a Chat Completion API response:

* `id:` the ID of the request
* `object: `the type of object returned (e.g., chat.completion)
* `created:` the timestamp of the request
* `model:` the full name of the model used to generate the response
* `usage:` the number of tokens used to generate the replies, counting prompt, completion, and total
* `choices: `a list of completion objects (only one, unless you set n greater than 1)
 * `message:` the message object generated by the model, with `role` and `content`
 * `finish_reason:` the reason the model stopped generating text (either stop, or length if max_tokens limit was reached)
 * `index: `the index of the specific completion in the list of the `choices` array

We can extract the reply as follows:

In [ ]:
print(response.choices[0].message.content)

You can make OpenAI output (such as ChatGPT responses) more readable in a Jupyter Notebook by displaying it as formatted Markdown using IPython's `display` functions.


In [ ]:
from IPython.display import Markdown

In [ ]:
display(Markdown(response.choices[0].message.content))

### **Some Prompting Tipps**

The next section provides some prompting tipps. For more details you can visit [here](https://platform.openai.com/docs/guides/prompt-engineering).

#### **Developer Messages**
Using developer messages you can provide context to a language model. This is essential because it helps guide the model's responses to align with the intended purpose or task. Here's why context matters:


* **Define the Model's Role:** The role the model plays is crucial in shaping its responses. Providing clear instructions about the model's purpose ensures that it stays on track with the task at hand. For example, when instructing the model to act as a professional translator, the message gives the model a clear sense of the expected response type (i.e., translation).

  *Example: "You are a professional translator. Translate the following technical document from English to Spanish, maintaining the technical accuracy and tone of the original content."*

* **Set Behavioral Guidelines:** Setting behavioral guidelines helps shape the tone, style, and structure of the model's response. If you want a concise and professional tone, the developer message will help the model recognize the importance of brevity and clarity.

  *Example: "Provide concise, professional responses in bullet points. Limit each bullet point to one sentence, and make sure each response starts with a key phrase or action."*

* **Provide Background Context:** Background context is incredibly important when tackling more complex tasks. It ensures the model understands the framework and scope of the subject matter, leading to responses that are grounded in the right knowledge.

  *Example: "Explain technical topics in simple terms for a beginner audience. Assume the reader has no prior knowledge of programming concepts. Provide analogies where necessary."*

* **Ensure Task-Specific Focus:** Keeping the model focused on a specific task avoids unnecessary tangents. By framing the interaction carefully, you help the model narrow the scope to what's relevant to the user.

  *Example: "You are a travel guide. Recommend 3 activities in Paris for a family with kids. Each activity should be family-friendly, affordable, and have a brief description."*

* **Reduce Ambiguity:** Reducing ambiguity means avoiding vague or unclear instructions, which ensures that the model understands exactly what is required. This leads to responses that are focused and relevant.

  *Example: "Write a Python function to efficiently calculate the Fibonacci sequence. Comment the code liberally to explain each step. Ensure that your code handles edge cases like negative numbers and large inputs gracefully."*

* **Define Output Format:** Specifying how the response should be structured helps the model to organize its answer in a way that is easily digestible and meets the user’s needs.

  *Example: Specify the output format clearly for the model to follow. The response should be structured as follows:*
  
  1. *Introduction: Provide a concise overview of the topic in 2-3 sentences, focusing on the key point.*

  2. *Main Body: Break the content down into 2-4 main sections, each with a clear heading. Each section should contain bullet points or short paragraphs (3-4 sentences per point).*
  
  3. *Conclusion: Wrap up the response with a short summary (1-2 sentences) highlighting the key takeaways. Ensure the tone is informative, to-the-point, and free of jargon. Keep the total response under 300 words.*


In [ ]:
#The developer message (with the "role" set to "developer") defines the model’s behavior,
# its role, and how it should approach the task. This message is responsible for setting the
# tone and framework within which the model operates.

#The user message (with the "role" set to "user") provides the specific task or question that
#the user wants the model to address. This is the content that the model will work on and respond to.

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "developer", "content": "You are a friendly and approachable computer science teacher, who explains complex topics in simple, child-friendly terms. Your goal is to make topics fun and easy to understand for young children."},
    {"role": "user", "content": "Explain generative AI in a fun and simple way, using examples that a 6-year-old would understand. Avoid technical jargon and make sure it’s engaging with a relatable analogy or story."},
  ]
)

display(Markdown(response.choices[0].message.content))


In [ ]:
#The developer message provides instructions to the model on how it should behave during the tast.
#It sets the tone ("humorous," "lighthearted," "playful") and defines the task ("write a fun, engaging article about hiking").
#It guides the model in terms of style (humorous, full of puns, relatable to beginners

#The user message contains the actual request or task that the user wants the model to address. It tells the model what to do with the context provided by the developer message.

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "developer", "content": "You are a humorous AI with a lighthearted and playful tone. Your task is to write a fun, engaging article about hiking that is entertaining, full of puns, and relatable for beginners."},
    {"role": "user", "content": "Can you write a short, fun article explaining why hiking is great for beginners? The article should be lighthearted, easy to understand, and highlight key benefits like health, mental clarity, and simple steps to get started."},
  ]
)

display(Markdown(response.choices[0].message.content))


In [ ]:
#Developer message
#Role Definition: teacher addressing an executive MBA audience. This means the model needs to provide a professional, clear, and concise explanation suitable for business-oriented students with an advanced level of education.
#Task Instructions: It outlines the specific tasks the model should cover:
#-Explain ML APIs (e.g., Google Cloud Vision, ChatGPT).
#-xplain open-source, off-the-shelf pre-trained models.
#-Compare ML APIs and open-source models.
#Tone & Focus: The message sets expectations for a clear, educational tone, appropriate for the context of a class aimed at business professionals.

#User message
#Task Request: The user message asks the model to explain the topics (ML APIs, open-source models, and their comparison) for the class
#Clarification: It doesn't need to specify the format, tone, or approach since the developer message already set those.
#The user message is mainly confirming the need for an explanation without adding additional constraints or details.

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "developer", "content": "You are a teacher teaching executive MBA students. 1) Explain what ML APIs such as Google Cloud Vision or Chat GPT are 2) Explain what open-source, off-the shelf pre-trained models are. 3) Compare the usage of ML APIs versus open-source, off-the shelf pretrained models."},
    {"role": "user", "content": "Can you provide an explanation of these topics for my class?"},
  ]
)

display(Markdown(response.choices[0].message.content))

In [ ]:
#Developer message
#Defines the Role: it specifies that the model should act as a teacher teaching executive MBA students.
#Sets the Context: it asks the model to explain ML APIs (like Google Cloud Vision or ChatGPT), open-source models, and to compare the two.
#Guides Style & Structure: it provides guidelines on how to deliver the content, such as making the explanation clear, concise, and using real-world examples.

#User message
#Specifies the Task:  it asks the model to explain the topics for their class.
#Details the Output Format: it asks for the explanation to be detailed, engaging, and suitable for an MBA-level audience.
#Focuses on Specific Outcomes: it asks for business-relevant examples and mentions that the comparison should be made between ML APIs and open-source models.

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "developer", "content": "You are a teacher explaining advanced topics to executive MBA students. Your explanation should be clear, concise, and use **real-world examples** where possible. 1) Explain what ML APIs (such as Google Cloud Vision or ChatGPT) are, including their use cases and how businesses leverage them. 2) Explain what open-source, off-the-shelf pre-trained models are, with an emphasis on how they differ from ML APIs. 3) Compare the usage of ML APIs versus open-source pre-trained models, highlighting the pros and cons of each in a business context."},
    {"role": "user", "content": "Can you provide a detailed explanation of these topics for my class? Please make the explanation clear, engaging, and appropriate for an MBA-level audience. It would be helpful to include business-relevant examples and highlight the practical implications of using ML APIs vs. open-source models."},
  ]
)

display(Markdown(response.choices[0].message.content))


#### **Few-shot Prompting**

**Few-shot prompting** is a technique where you provide a language model with a **small number of examples** (typically 2-5) within the prompt to demonstrate the desired format, style, or behavior for a specific task.

In the given example, the model is shown several pairs of corporate jargon statements and their plain English translations. These examples help the model understand the task's context and expected response style without explicitly programming the logic. By seeing the pattern of input-output pairs, the model can generalize and generate appropriate responses for new inputs, such as translating "This late pivot means we don't have time to boil the ocean for the client deliverable." into "This last-minute change means we can't spend excessive time on the client's project". Few-shot prompting leverages the model's ability to infer rules and structures from minimal examples, making it a powerful and flexible tool for a wide range of tasks.







In [ ]:
# An example of a faked few-shot conversation to prime the model into translating business jargon to simpler speech
# Example adapted from: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_format_inputs_to_ChatGPT_models.ipynb

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": "You are a helpful assistant specializing in translating corporate jargon into simple, plain English."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)

To help clarify that the example messages are not part of a real conversation, and shouldn't be referred back to by the model, you can try setting the `name` field of `system` messages to `example_user` and `example_assistant`.


In [ ]:
# The business jargon translation example, but with example names for the example messages
# Example adapted from: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_format_inputs_to_ChatGPT_models.ipynb

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English."},
        {"role": "developer", "name":"example_user", "content": "New synergies will help drive top-line growth."},
        {"role": "developer", "name": "example_assistant", "content": "Things working well together will increase revenue."},
        {"role": "developer", "name":"example_user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "developer", "name": "example_assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)

In [ ]:
# description: classify titles into categories

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": "You are a helpful, pattern-following assistant that classifies titles into categories."},
        {"role": "developer", "name": "example_user", "content": "Paris auf der Saslong nicht zu biegen"},
        {"role": "developer", "name":"example_assistant", "content": "Ski Alpin"},
        {"role": "developer", "name":"example_user", "content": "Kobayashi geht in Führung"},
        {"role": "developer", "name":"example_assistant", "content": "Skispringen"},
        {"role": "developer", "name": "example_user", "content": "Hütter fährt in Abfahrt aufs Podest"},
        {"role": "developer", "name":"example_assistant", "content": "Ski Alpin"},
        {"role": "developer", "name": "example_user", "content": "Schweizer Freudentag: Flury brilliert vor Hählen"},
        {"role": "developer", "name":"example_assistant", "content": "Ski Alpin"},
        {"role": "developer", "name": "example_user", "content": "Arsenal unterliegt Tottenham im Derby"},
        {"role": "developer", "name":"example_assistant", "content": "Fussball"},
        {"role": "user", "content": "Seoanes Gladbach muss sich mit Remis begnügen"},
        #{"role": "user", "content": "Superadler Kraft: Endlich einmal gelbe Weihnachten"},

    ],

    temperature=0,
)

print(response.choices[0].message.content)


### **Customizing Chat Completions for Targeted Tasks**


In the next section, we will explore how to **customize Chat Completions for targeted tasks**. This involves creating **custom functions** that use a static system message while dynamically passing the user message as a parameter. This approach allows us to maintain control over the context provided to the model while tailoring the input to specific needs.

In Lab07, we will build on this concept by using LangChain's Prompt Templates to achieve similar functionality. Prompt Templates enable us to define reusable prompts with placeholders, effectively managing both static and dynamic elements. This enhances flexibility, modularity, and efficiency in prompt design.

#### **Using Chat Completions for Text Translation**

The next section provides an example of a custom function for text translation. In this example, the system message sets the context for the model, while the text to be translated is dynamically passed as a parameter.

In [ ]:
def translateFromEnglishToGerman(text):
  response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "developer",
      "content": "You will be provided with a sentence in English, and your task is to translate it into German."
    },
    {
      "role": "user",
      "content": text
    }
  ],
  temperature=0.7,
  )
  return response.choices[0].message.content

In [ ]:
translatedText = translateFromEnglishToGerman("My name is Barbara. What is yours?")

In [ ]:
print(translatedText)

#### **Using Chat Completions for Image Prompt Generation**

In [ ]:
def createImagePrompt(text):
  response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
     {
        "role": "developer",
        "content": "You are a skillful prompt generator that refines text for an image prompt"
    },
    {
        "role": "user",
        "content": text},
  ]
  )
  return response.choices[0].message.content

In [ ]:
imagePrompt = createImagePrompt("nice and cinematic mountain ranges")

In [ ]:
display(Markdown(imagePrompt))

## **Text to Speech Generation**

The Audio API provides a `speech` endpoint based on our TTS (text-to-speech) model. It comes with 6 built-in voices (`alloy`, `echo`, `fable`, `onyx`, `nova`, and `shimmer`) and can be used to:

* Narrate a written blog post
* Produce spoken audio in multiple languages
* Give real time audio output using streaming

To check out supported languages visit https://platform.openai.com/docs/guides/text-to-speech.

### **Audio Generation Example**

In [ ]:
def generateTextToSpeech(text):
  speech_file_path = "speech.mp3"
  response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=text
  )

  response.stream_to_file(speech_file_path)
  return speech_file_path

In [ ]:
import IPython.display as ipd
fileName=generateTextToSpeech("Today is a wonderful day to build something people love!")
ipd.Audio(filename=fileName)

In the next code snippet there is a translation step into the target language (e.g., German) before the audio output is generated.

In [ ]:
# Here we create a whole simple pipeline starting from translation and then speech generation

translatedText = translateFromEnglishToGerman("My name is Barbara. What is yours?")

fileName=generateTextToSpeech(translatedText)
ipd.Audio(filename=fileName)

## **Speech to Text (Audio API)**

The Audio API provides two speech to text endpoints, `transcriptions` and `translations`, based on our state-of-the-art open source large-v2 Whisper model. They can be used to:

* Transcribe audio into whatever language the audio is in.
* Translate and transcribe the audio into english.

File uploads are currently limited to 25 MB and the
following input file types are supported: `mp3`, `mp4`, `mpeg`, `mpga`, `m4a`, `wav`, and `webm`.

### **Transcription Example**

In [ ]:
def transcribeAudio(fileName):
  audio_file = open(fileName, "rb")
  transcript = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    response_format="text"
  )
  return transcript

You will now be asked to upload an .mp3 file which will then be transcribed.

To explore the transcription capabilities of the Audio API you can experiment with one of the news audio files available at https://www.srf.ch/audio/nachrichten.

In [ ]:
!wget -O Podcast.mp3 https://download-media.srf.ch/world/audio/4x4_Podcast_radio/2025/01/4x4_Podcast_radio_AUDI20250128_NR_0038_3c81668cc4664fff9a26020d4eb47f0a.mp3?d=ap&assetId=e0ed83a1-efc1-373c-b4aa-9878c79f2e31


In [ ]:
# The file name which is passed as a parameter to the function transcribeAudio needs to correspond to the file you have uploaded
transcribedAudio= transcribeAudio("Podcast.mp3")
display(Markdown(transcribedAudio))

Now let's create a bullet list with the news items covered.

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "developer", "content": "You are a helpful news summarizer. Create a bulletlist of the different news topics covered with 1-2 sentences per item in German."},
    {"role": "user", "content": "You are provided with a transcript of a news Podcast." + transcribedAudio},
  ]
)

summary = response.choices[0].message.content
display(Markdown(summary))

Now let's use a prompt to create a summary of the news podcast just focussing on topics we are intersted in.

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "developer", "content": "You are a helpful assistant specialized on creating concise and accurate summaries on KI topics. Provide a max 20 sentence summary only covering KI topics in German."},
    {"role": "user", "content": "You are provided with a transcript of a news Podcast." + transcribedAudio},
  ]
)

summary = response.choices[0].message.content
display(Markdown(summary))

Now lets use the `generateTextToSpeech` function we implemented before to create an audio with the summary.

In [ ]:
fileName=generateTextToSpeech(summary)
ipd.Audio(filename=fileName)

Should German not be your mother tongue, then you can use the `translations` endpoint to transcribe the file into English.

In [ ]:
def transcribeAndTranslateAudio(fileName):
  audio_file = open(fileName, "rb")
  transcript = client.audio.translations.create(
    model="whisper-1",
    file=audio_file,
    response_format="text"
  )
  return transcript

In [ ]:
display(Markdown(transcribeAndTranslateAudio("Podcast.mp3")))

### **Working with Audio Segments**

By default, the Whisper API only supports files that are less than 25 MB. To work with larger audio files, you will you will need to break them into chunks.

One option is to use the **PyDub open source Python package** to split the audio.

In [ ]:
!pip install -q pydub

In [ ]:
# code adapted from: https://platform.openai.com/docs/guides/speech-to-text/longer-inputs

from pydub import AudioSegment

def segmentAudio(audioFile):
  song = AudioSegment.from_mp3(audioFile)
  # PyDub handles time in milliseconds
  two_minutes = 2 * 60 * 1000
  first_2_minutes = song[:two_minutes]
  first_2_minutes.export("TwoMinutes" + audioFile, format="mp3")
  return "TwoMinutes" + audioFile

You will be asked to upload a file to be transcribed.

To explore the segmentation capabilities of the Audio API you can experiment with one longer audio files such as https://www.srf.ch/audio/tagesgespraech.

In [ ]:
!wget -O Tagesgespraech_18-01-2025.mp3 "https://download-media.srf.ch/world/audio/Tagesgespraech_radio/2025/01/Tagesgespraech_radio_AUDI20250118_NR_0030_bf244ffb90af4470abd2cebd91107e91.mp3?d=ap&assetId=641aa6a1-597f-3c50-bce5-a8e0193e76cc"

We can then play the split audio file.

In [ ]:
# The file name which is passed as a parameter to the function transcribeAudio needs to correspond to the file you have uploaded
fileName = "Tagesgespraech_18-01-2025.mp3"
twoMinuteFile = segmentAudio(fileName)
ipd.Audio(filename=twoMinuteFile)

.. and even translate it to another language.

In [ ]:
display(Markdown(transcribeAndTranslateAudio(twoMinuteFile)))

To experiment a bit more with Whisper visit [here](https://github.com/openai/openai-cookbook/blob/main/examples/Whisper_processing_guide.ipynb). Moreover, if you want to learn more on prompting with Whisper visit [here](https://github.com/openai/openai-cookbook/blob/main/examples/Whisper_prompting_guide.ipynb).

## **Image Generation**



The Images API provides three methods for interacting with images:

* Creating images from scratch based on a text prompt (DALL·E 3 and DALL·E 2)
* Creating variations of an existing image (DALL·E 2 only)
* Creating edited versions of images by having the model replace some areas of a pre-existing image, based on a new text prompt (DALL·E 2 only)


The image generations endpoint allows you to create an original image given a text prompt. When using DALL·E 3, images can have a size of 1024x1024, 1024x1792 or 1792x1024 pixels.

By default, images are generated at `standard` quality, but when using DALL·E 3 you can set quality: "`hd`" for enhanced detail. Square, standard quality images are the fastest to generate.

You can request 1 image at a time with DALL·E 3 (request more by making parallel requests) or up to 10 images at a time using DALL·E 2 with the n parameter.

### **Creating Images From Text**

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.images.generate(
  model="dall-e-3",
  prompt="a downhill skier in the swiss alps",
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url

In [ ]:
# Here we take the URL created previously and open the image
from PIL import Image
import urllib.request

with urllib.request.urlopen(image_url) as url:
    img=Image.open(url)
    display(img)